In [37]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
import pandas as pd

In [231]:
driver = webdriver.Chrome('/Users/AngelaChi/Desktop/chromedriver')
base_url = 'https://tripadvisor.com'
driver.get(base_url)
driver.find_element_by_xpath('//*[@id="taplc_trip_search_home_default_0"]/div[2]/div[1]/div/span/input').send_keys('Grand Canyon National Park')
driver.find_element_by_xpath('//*[@id="SUBMIT_HOTELS"]').click()
time.sleep(2)
k = 0
res2 = []
driver2 = webdriver.Chrome('/Users/AngelaChi/Desktop/chromedriver')
while k < 15:
    if len(res2) > 20000:
        break
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    hotels = soup.findAll('a', {'class':'property_title'})
    print len(hotels)
    b = 0
    while b < len(hotels):
        if len(res2) > 20000:
            break
        driver2.get(base_url + str(hotels[b]['href']))
        html2 = driver2.page_source
        soup2 = BeautifulSoup(html2, 'html.parser')
        try:
            link = base_url+str(soup2.find('div',{'class':'quote isNew'}).find('a')['href'])
        except:
            link = base_url+str(soup2.find('div',{'class':'quote'}).find('a')['href'])
        driver2.get(link)
        review_num = int(re.sub(r'[^0-9]+','',driver2.find_element_by_xpath('//*[@id="REVIEWS"]/div[2]/div[1]/span[2]').text[1:-1]))
        if review_num < 5000 and review_num > 2000:
            html2 = driver2.page_source
            soup2 = BeautifulSoup(html2, 'html.parser')
            num = soup2.find('a',{'class','pageNum last taLnk '}).text
            j = 0
            while j < int(num)-1:
                if len(res2) > 20000:
                    break
                try:
                    reviews = soup2.findAll('div',{'class':'ui_column is-9'})
                    a = 0
                    while a < len(reviews):
                        title = str(reviews[a].find('span',{'class':'noQuotes'}).text)
                        text = str(reviews[a].find('p').text)
                        overall_rate = str(reviews[a].find('div', {'class':'rating reviewItemInline'}).find('span')['class'][1][-2])
                        attributes = reviews[a].find('ul',{'class','recommend'})
                        d = {'Value':None, 'Location':None, 'Sleep Quality':None,'Rooms':None,'Cleanliness':None,'Service':None}
                        if attributes == None:
                            pass
                        else:
                            for li in attributes.findAll('li',{'recommend-answer'}):
                                d[li.text] = str(li.find('div')['class'][1])[-2]

                        info = [title,text,overall_rate,d]
                        res2.append(info)
                        if len(res2) > 20000:
                            break
                        a+=1
                except:
                    pass
                j+=1
                time.sleep(1)
                driver2.find_element_by_link_text('Next').click()
                html2 = driver2.page_source
                soup2 = BeautifulSoup(html2, 'html.parser')
                time.sleep(2)
        b+=1
         #driver2.close()
    driver.find_element_by_link_text(str(k+2)).click()
    k+=1

60


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"2"}
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.33.506106 (8a06c39c4582fbfbab6966dbb1c38a9173bfb1a2),platform=Mac OS X 10.10.5 x86_64)


In [233]:
print (k,a,j,b)

(0, 1, 239, 37)


In [232]:
len(res2)

20001

In [235]:
data2 = pd.DataFrame(res2,columns = ['title','text','overall_rate','attributes'])

In [236]:
ls = data2['attributes'].values.tolist()
v2 = []
for i in range(len(ls)):
    v2.append([ls[i]['Cleanliness'],ls[i]['Location'],ls[i]['Rooms'],ls[i]['Service'],ls[i]['Sleep Quality'],ls[i]['Value']])

In [237]:
v3 = []
for i in range(len(res2)):
    v3.append(res2[i][:-1]+v2[i])

In [239]:
data_gc = pd.DataFrame(v3,columns = ['title','text','overall_rate','Cleanliness','Location','Rooms','Service','Sleep Quality','Value'])

In [240]:
data_gc.shape

(20001, 9)

In [241]:
data_gc = data_gc.drop_duplicates()

In [242]:
data_gc.shape

(11073, 9)

In [243]:
data_gc.to_csv('gc_new.csv', index = False)